In [1]:
import pandas as pd
import os
from dshortcuts.utils import open_pickle, save_pickle

In [2]:
os.listdir('../input')

['test.csv',
 'sample_submission.csv',
 'item_categories.csv',
 'trainDtypes.pickle',
 'sales_train.csv',
 'itemCategoriesDtypes.pickle',
 'items.csv',
 'shops.csv']

In [3]:
# open prepared dtypes to save memory
trainDtypes = open_pickle(os.path.join('../input', 'trainDtypes.pickle'))
itemCategoriesDtypes = open_pickle(os.path.join('../input', 'itemCategoriesDtypes.pickle'))

In [4]:
train = pd.read_csv('../input/sales_train.csv', dtype=trainDtypes, parse_dates=['date'])
itemCategories = pd.read_csv('../input/item_categories.csv', dtype=itemCategoriesDtypes)
test = pd.read_csv('../input/test.csv')
shops = pd.read_csv('../input/shops.csv')
items = pd.read_csv('../input/items.csv')

In [5]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.000000,1.0
1,2013-03-01,0,25,2552,899.000000,1.0
2,2013-05-01,0,25,2552,899.000000,-1.0
3,2013-06-01,0,25,2554,1709.050049,1.0
4,2013-01-15,0,25,2555,1099.000000,1.0


##### Categories
1. Are they broad enough?
2. Are they descriptive enough?

In [11]:
itemCategories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
...,...,...
79,Служебные,79
80,Служебные - Билеты,80
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82


too descriptive. At first look
we may try to decouple broad categories by split('-')`

In [6]:
train = train.merge(items.drop('item_name', axis=1), on='item_id', how='left')

#### Obvious features: 

- day
- month
- year
- dayofweek
- is workingday
- delta from every 22th day of month (~salary day in Rus)
- delta from every 7th day of month (~pre-salary day in Rus)

#### Sales features: (think more...)

- monthly sales by {shop_id, category_id}
- monthly sales by {shop_id, item_id}
- monthly sales by {category_id}

#### Geographical features
1. Get the city from the shop info
2. Try to get the lat lon of the shop with map api ?

-------------

- what's next?

In [7]:
train.groupby(['date_block_num', 'shop_id', 'item_id']).item_cnt_day.sum()

date_block_num  shop_id  item_id
0               0        32         6.0
                         33         3.0
                         35         1.0
                         43         1.0
                         51         2.0
                                   ... 
33              59       22087      6.0
                         22088      2.0
                         22091      1.0
                         22100      1.0
                         22102      1.0
Name: item_cnt_day, Length: 1609124, dtype: float16

In [23]:
valid = train[train['date_block_num'] == (train['date_block_num'].max())]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
2882335,2015-10-23,33,45,13315,649.0,1.0,47
2882336,2015-05-10,33,45,13880,229.0,1.0,55
2882337,2015-02-10,33,45,13881,659.0,1.0,55
2882338,2015-12-10,33,45,13881,659.0,1.0,55
2882339,2015-04-10,33,45,13923,169.0,1.0,40


In [20]:
(test['shop_id'].isin(train['shop_id']) == False).sum(), \
(test['item_id'].isin(train['item_id']) == False).sum()

(0, 15246)

In [25]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
